In [11]:
import pickle

### Set up environment

# import packages
import pandas as pd
import numpy as np
import os
from glob import iglob
#import matplotlib.pyplot as plt
#import seaborn as sns
import random
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
#from matplotlib.pyplot import figure
from sklearn.neural_network import MLPRegressor
import glob

import json
import csv

from sqlalchemy.dialects.mssql.information_schema import columns

In [12]:
def normalise_to_296(dataframe):
    temp = dataframe['T']
    n = dataframe['n']
    gamma_temp = dataframe['gamma']
    gamma_uncertainty = dataframe['gamma_uncertainty']

    gamma_296 = gamma_temp * (temp / 296)**n

    rel_err2 = (gamma_uncertainty / gamma_temp)**2 + (np.log(temp / 296) * 0.1)**2
    gamma_uncertainty_296 = gamma_296 * np.sqrt(rel_err2)

    dataframe['gamma'] = gamma_296.round(3)
    dataframe['gamma_uncertainty'] = gamma_uncertainty_296.round(6)
    dataframe['T'] = 296

    return dataframe

In [13]:
# file = 'MAST'
file = 'GUEST'


In [14]:
if file == 'MAST':
    file_mast = pd.read_excel('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/'+'Using LLMs to scrape data/Data_masters_student/data_Pengxia_CORRECTED.xlsx', sheet_name=None)#, dtype={'J':np.float64, 'Jpp':np.float64, 'gamma':np.float64, 'gamma_uncertainty':np.float64}
elif file == 'GUEST':
    file_mast = pd.read_excel('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/'+'Other_broadeners/References/Guest_LLM_data.xlsx', sheet_name=None)#, dtype={'J':np.float64, 'Jpp':np.float64, 'gamma':np.float64, 'gamma_uncertainty':np.float64}

In [15]:
count = 0
list_of_sheets = []
for name, sheet in file_mast.items():
    print(name)
    if name == "Audit Summary":
        continue
    if name.split(' ')[-1] == 'PROBLEM':
        continue
    if name.split(' ')[-1] == 'ELEC':
        continue
    if name.split(' ')[1] in ['K', 'Rb']:
        continue

    if file == 'MAST':
        sheet = sheet[['active_species', 'perturber', "J'", "J''", 'gamma', 'gamma_units', 'gamma_unc_abs', 'n', 'temperature_k', 'line_profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
        sheet = sheet.set_axis(['molecule', 'broadener', 'J', 'Jpp', 'gamma', 'gamma_units', 'gamma_uncertainty', 'n', 'T', 'profile', 'K', 'Ka_aprox', 'Kc_aprox', 'Kpp', 'Kapp_aprox', 'Kcpp_aprox'], axis=1)
        sheet['paper'] = name.split(' ')[0]

    elif file == 'GUEST':
        sheet = sheet[['active species', 'perturbing species', "J'", "Ka'", "Kc'", 'J"', 'Ka"', 'Kc"', 'gamma', 'percentage error in gamma', 'absolute uncertainty in gamma', 'n', 'temperature (K)', 'line profile', 'notes']]
        sheet = sheet.set_axis(['active species', 'perturbing species', "J'", "Ka'", "Kc'", 'J"', 'Ka"', 'Kc"', 'gamma', 'percentage error in gamma', 'absolute uncertainty in gamma', 'n', 'temperature (K)', 'line profile', 'notes'], axis=1)
        sheet = sheet.rename(columns={'line profile': 'profile', 'temperature (K)': 'T', 'absolute uncertainty in gamma': 'gamma_uncertainty', 'active species': 'molecule', 'perturbing species': 'broadener', "J'": 'J', "Ka'": 'Ka_aprox', "Kc'": 'Kc_aprox', 'J"': 'Jpp', 'Ka"': 'Kapp_aprox', 'Kc"': 'Kcpp_aprox'})
        sheet['paper'] = name.split(' ')[0]
        sheet = sheet.replace('uncertain', np.nan)


    list_of_sheets.append(sheet)


303 HBr by CO2
304 HF by CO2
307 N2 by H2
308 N2 by H2
310 N2 by CO2
311 N2 by CO
312 HCN by N2
313 H2O by He
316 CO by H2
317 CO by air and H2
318 CO by O2, N2, H2, He, CO2, 
319 H2O by H2 and He
321 H2O by H2
322 H2O by H2 and He
324 13C12CH4 by self
325 13CO and C18O by air
327 CH4 by C3H8 and C4H10
328 H2O by He, Ar, Kr
329 CO by CO2
330 C2H2 by CO2
335 H2O by O2
336 H2O by He
337 H2O by CO2
339 H2O by Ar
340 H2O by Ar, He, Ne, Kr
341 H2S by self, H2O, N2, O2, H
343 H2O by He
344 H2O–H2O, NH3–NH3, DCl–HCl a
345 H2S by He, Ne, Ar, Kr, Xe
346 H2S by He
347 NH3 by Ar and He
348 H2S by He, Ne, Ar, Kr, Xe
349 H2O by He, Ar, Kr
350 NH3 by Ar and He
351 H2O by N2, Ar and He
352 H2O by Ne, Kr and Xe
354 H2O by Ne, Kr and Xe
355 H2O by Ar, Kr, H2 and He
357 H2O by CO2
358 H2O by He, Ar, Kr and Xe
360 H2O by H2
361 H2O by He
363 H2O by Ar
365 H2O by H2
366 SO2 by CO2
367 H2O by Kr
368 H2O by Kr
370 H2O by Xe
371 H2O by CO2
372 H2O by Xe
373 H2O by CO2
374 H2O by CO2
375 HDO by CO2
47 NO by H

In [16]:
masters_data = pd.concat(list_of_sheets).reset_index(drop=True)

In [17]:
print(masters_data['profile'].unique())

['Lorentzian' nan 'Voigt' 'qSDVP (quadratic speed-dependent Voigt)'
 'speed-dependent Voigt with line mixing'
 'qSDV (quadratic speed-dependent Voigt)' 'qSDV' 'HTP'
 'hard-collision (Rautian–Sobelman)' 'hard/soft collision (Dicke)' 'SDVP'
 'quadratic Voigt (SDV)' 'speed-dependent Voigt' 'QSDVP' 'qSDV Voigt'
 'quadratic speed-dependent Voigt (qSDV)' 'Rautian (Dicke-narrowed)'
 'qSDHCP (HTP)']


In [18]:
masters_data['profile'] = masters_data['profile'].fillna('Voigt')
masters_data = masters_data[(masters_data['profile'] == 'Voigt') | (masters_data['profile'] == 'Lorentzian')]
masters_data['profile'] = 'Voigt'

In [19]:
masters_data

,molecule,broadener,J,Ka_aprox,Kc_aprox,Jpp,Kapp_aprox,Kcpp_aprox,gamma,percentage error in gamma,gamma_uncertainty,n,T,profile,notes,paper
0,H79Br,HBr,1.0,NaN,NaN,0.0,NaN,NaN,0.1318,5.0,0.006590,NaN,296.0,Voigt,Self-broadening parameter from Table I at 296 ...,303
1,H81Br,HBr,1.0,NaN,NaN,0.0,NaN,NaN,0.1318,5.0,0.006590,NaN,296.0,Voigt,Self-broadening parameter from Table I at 296 ...,303
2,H79Br,HBr,2.0,NaN,NaN,1.0,NaN,NaN,0.1348,5.0,0.006740,NaN,296.0,Voigt,Self-broadening parameter from Table I at 296 ...,303
3,H81Br,HBr,2.0,NaN,NaN,1.0,NaN,NaN,0.1338,5.0,0.006690,NaN,296.0,Voigt,Self-broadening parameter from Table I at 296 ...,303
4,H79Br,HBr,3.0,NaN,NaN,2.0,NaN,NaN,0.1381,5.0,0.006905,NaN,296.0,Voigt,Self-broadening parameter from Table I at 296 ...,303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,CO,Ar,22.0,NaN,NaN,21.0,NaN,NaN,0.0388,10.0,0.003880,NaN,298.0,Voigt,|m|=21; Table 2 experimental values,389
5868,CO,Ar,23.0,NaN,NaN,22.0,NaN,NaN,0.0382,10.0,0.003820,NaN,298.0,Voigt,|m|=22; Table 2 experimental values,389
5869,CO,Ar,24.0,NaN,NaN,23.0,NaN,NaN,0.0377,10.0,0.003770,NaN,298.0,Voigt,|m|=23; Table 2 experimental values,389
5870,CO,Ar,25.0,NaN,NaN,24.0,NaN,NaN,0.0372,10.0,0.003720,NaN,298.0,Voigt,|m|=24; Table 2 experimental values,389


In [21]:
# enforce plausible range
masters_data.loc[(masters_data["n"] < 0) | (masters_data["n"] > 1), "n"] = np.nan

# replace remaining NaN with default
masters_data["n"] = masters_data["n"].fillna(0.5)

In [22]:
masters_data['T'] = masters_data['T'].astype(float)

In [23]:
masters_data = masters_data[(masters_data['T'] < 500) & (masters_data['T'] > 100)]

In [24]:
masters_data['gamma_uncertainty'] = masters_data['gamma_uncertainty'].fillna(masters_data['gamma']*0.5)

/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/1246812276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masters_data['gamma_uncertainty'] = masters_data['gamma_uncertainty'].fillna(masters_data['gamma']*0.5)


In [25]:
masters_data['gamma_uncertainty'] = masters_data['gamma_uncertainty'].fillna(masters_data['gamma']*0.5)


/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/3384779290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masters_data['gamma_uncertainty'] = masters_data['gamma_uncertainty'].fillna(masters_data['gamma']*0.5)


In [26]:
masters_data['molecule'].unique()

array(['H79Br', 'H81Br', 'HF', 'N2', 'H2O', 'CO', '13C12CH4', 'C2H4',
       'CH4', 'C2H2', 'H2S', 'NH3', 'NO', 'HDO', 'HCl', 'CO2'],
      dtype=object)

In [27]:
masters_data['broadener'].unique()


array(['HBr', 'CO2', 'H2', 'CO', 'He', 'air', 'N2', 'O2', '13C12CH4',
       'C2H4', 'CH4', 'C3H8', 'n-C4H10', 'i-C4H10', 'Ar', 'Kr', 'Ne',
       'Xe', 'H2O', 'NO', 'HDO'], dtype=object)

In [28]:
if file == 'MAST':
    prolate_data = masters_data[masters_data['molecule'].isin(['CH3F', 'CH3Br', 'CH3-35Cl', 'CH3D', 'CH3I', 'CH3OH', 'CF3CCH', '(CH3)3CCN'])]

    na_data = prolate_data[prolate_data['K'].isna()]
    na_data['K'] = na_data['J']
    na_data['Kpp'] = na_data['Jpp']
    prolate_data[prolate_data['K'].isna()] = na_data

    prolate_data['Ka_aprox'] = prolate_data['K']
    prolate_data['Kapp_aprox'] = prolate_data['Kpp']
    prolate_data['Kc_aprox'] = prolate_data['J'] - prolate_data['K']
    prolate_data['Kcpp_aprox'] = prolate_data['Jpp'] - prolate_data['Kpp']

    masters_data[masters_data['molecule'].isin(['CH3F', 'CH3Br', 'CH3-35Cl', 'CH3D','CH3I', 'CH3OH', 'CF3CCH', '(CH3)3CCN'])] = prolate_data

In [29]:
 if file == 'MAST':
    oblate_data = masters_data[masters_data['molecule'].isin(['NH3', 'CHF3', 'CH3'])]

    na_data = oblate_data[oblate_data['K'].isna()]
    na_data['K'] = na_data['J']
    na_data['Kpp'] = na_data['Jpp']
    oblate_data[oblate_data['K'].isna()] = na_data

    oblate_data['Ka_aprox'] = oblate_data['J'] - oblate_data['K']
    oblate_data['Kapp_aprox'] = oblate_data['Jpp'] - oblate_data['Kpp']
    oblate_data['Kc_aprox'] = oblate_data['K']
    oblate_data['Kcpp_aprox'] = oblate_data['Kpp']

    masters_data[masters_data['molecule'].isin(['NH3', 'CHF3', 'CH3'])] = oblate_data

 elif file == 'GUEST':
    oblate_data = masters_data[masters_data['molecule'].isin(['NH3', 'CHF3', 'CH3'])]

    cna_data = oblate_data[oblate_data['Kc_aprox'].isna()]
    cna_data['Kc_aprox'] = cna_data['J']
    cna_data['Kcpp_aprox'] = cna_data['Jpp']
    oblate_data[oblate_data['Kc_aprox'].isna()] = cna_data

    ana_data = oblate_data[oblate_data['Ka_aprox'].isna()]
    ana_data['Ka_aprox'] = ana_data['J'] - ana_data['Kc_aprox']
    ana_data['Kapp_aprox'] = ana_data['Jpp'] - ana_data['Kcpp_aprox']
    oblate_data[oblate_data['Ka_aprox'].isna()] = ana_data

    masters_data[masters_data['molecule'].isin(['NH3', 'CHF3', 'CH3'])] = oblate_data

/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/308002029.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cna_data['Kc_aprox'] = cna_data['J']
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/308002029.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cna_data['Kcpp_aprox'] = cna_data['Jpp']
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/308002029.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [30]:
spherical_data = masters_data[masters_data['molecule'].isin(['CH4', 'SF6'])]

spherical_data['Ka_aprox'] = spherical_data['J'] / 2
spherical_data['Kapp_aprox'] = spherical_data['Jpp'] / 2
spherical_data['Kc_aprox'] = spherical_data['J'] / 2
spherical_data['Kcpp_aprox'] = spherical_data['Jpp'] / 2

masters_data[masters_data['molecule'].isin(['CH4', 'SF6'])] = spherical_data

/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/622130489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spherical_data['Ka_aprox'] = spherical_data['J'] / 2
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/622130489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spherical_data['Kapp_aprox'] = spherical_data['Jpp'] / 2
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/622130489.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [31]:
linear_data = masters_data[masters_data['molecule'].isin(['NO', 'O2', 'HF', 'HCl', 'CO2', '13CO', 'CO', 'OH', '15NO', 'HD', 'HCN', 'N2', 'CS2', 'DCl', 'HBr', 'CN', 'HI', 'C2H2', 'DCl', 'H79Br', 'H81Br', 'CO'])]

linear_data['Ka_aprox'] = 0
linear_data['Kc_aprox'] = linear_data['J']
linear_data['Kapp_aprox'] = 0
linear_data['Kcpp_aprox'] = linear_data['Jpp']

masters_data[masters_data['molecule'].isin(['NO', 'O2', 'HF', 'HCl', 'CO2', '13CO', 'CO', 'OH', '15NO', 'HD', 'HCN', 'N2', 'CS2', 'DCl', 'HBr', 'CN', 'HI', 'C2H2', 'DCl', 'H79Br', 'H81Br'])] = linear_data

/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/2694348807.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_data['Ka_aprox'] = 0
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/2694348807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_data['Kc_aprox'] = linear_data['J']
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/2694348807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [32]:
remaining_na_data = masters_data[masters_data['Ka_aprox'].isna()]

remaining_na_data['Ka_aprox'] = round(remaining_na_data['J'] / 2)
remaining_na_data['Kapp_aprox'] = round(remaining_na_data['Jpp'] / 2)
remaining_na_data['Kc_aprox'] = round(remaining_na_data['J'] - 1)
remaining_na_data['Kcpp_aprox'] = round(remaining_na_data['Jpp'] / 2)

masters_data[masters_data['Ka_aprox'].isna()] = remaining_na_data

/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/1591101991.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masters_data[masters_data['Ka_aprox'].isna()] = remaining_na_data
/opt/anaconda3/envs/MLBDenv/lib/python3.8/site-packages/pandas/core/indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [33]:
reduced_data = masters_data.loc[masters_data['paper'].drop_duplicates().index]
reduced_data

,molecule,broadener,J,Ka_aprox,Kc_aprox,Jpp,Kapp_aprox,Kcpp_aprox,gamma,percentage error in gamma,gamma_uncertainty,n,T,profile,notes,paper
0,H79Br,HBr,1.0,0.0,1.0,0.0,0.0,0.0,0.131800,5.000000,0.006590,0.5000,296.0,Voigt,Self-broadening parameter from Table I at 296 ...,303
48,HF,CO2,3.0,0.0,3.0,4.0,0.0,4.0,0.065000,2.000000,0.001300,0.5000,296.0,Voigt,"P(4) line. γ = Av/P from Table II. J''=4,J'=3....",304
59,N2,H2,4.0,0.0,4.0,2.0,0.0,2.0,0.054000,6.000000,0.003240,0.5000,294.0,Voigt,Table I S-branch Raman linewidths; J'=J''+2; 2...,307
104,N2,H2,0.0,0.0,0.0,0.0,0.0,0.0,0.058200,20.618557,0.012000,0.5000,298.0,Voigt,Isotropic Raman Q-branch Q(0) of N2 broadened ...,308
164,N2,CO2,1.0,0.0,1.0,1.0,0.0,1.0,0.083400,3.957000,0.003300,0.5000,296.0,Voigt,VALUES SEE LINE MIXING - COULD BE PROBLEM? Q(1...,310
216,N2,CO,0.0,0.0,0.0,0.0,0.0,0.0,0.018070,7.690000,0.001390,0.5000,295.0,Voigt,Converted from 26 mK/atm using 1 mK = 6.95e-4 ...,311
255,H2O,He,3.0,1.0,3.0,2.0,2.0,0.0,0.031052,20.000000,0.006210,0.5000,120.0,Voigt,Total pressure-broadening coefficient g_if (in...,313
300,CO,H2,1.0,0.0,1.0,0.0,0.0,0.0,0.123500,3.120000,0.003800,0.5000,140.0,Voigt,"Table I (CO–H2 J'=1←J""=0). Linear CO so Ka',Kc...",316
320,CO,air,24.0,0.0,24.0,25.0,0.0,25.0,0.046500,50.000000,0.023250,0.7500,296.0,Voigt,"CO 1-0 band; m=-25 (P-branch, m=-J"" in P and m...",317
348,CO,N2,5.0,0.0,5.0,4.0,0.0,4.0,0.063453,0.264000,0.000167,0.7725,296.0,Voigt,Linear CO; Ka and Kc not defined. R(4) rotatio...,318


In [34]:
processed_data = normalise_to_296(masters_data)

/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/837221608.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['gamma'] = gamma_296.round(3)
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/837221608.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['gamma_uncertainty'] = gamma_uncertainty_296.round(6)
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_75830/837221608.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [35]:
if file == 'MAST':
    processed_data = processed_data.drop(['K', 'Kpp'], axis=1).reset_index(drop=True)

In [36]:
if file == 'MAST':
    processed_data.to_csv('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/Other_broadeners/Files_with_new_data/LLM_data/LLM_scraped_data.csv')
if file == 'GUEST':
    processed_data.to_csv('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/Other_broadeners/Files_with_new_data/LLM_data/Guest_LLM_scraped_data.csv')

In [101]:
import pickle

### Set up environment

# import packages
import pandas as pd
import numpy as np
import os
from glob import iglob
#import matplotlib.pyplot as plt
#import seaborn as sns
import random
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
#from matplotlib.pyplot import figure
from sklearn.neural_network import MLPRegressor
import glob

import json
import csv

# Set parameters
#T = 298      # Kelvin


# define Jeanna's broadening formalism for use later.  Taken from paper ...
def broadening(m, T, ma, mp, b0):
    gamma = 1.7796e-5 * (m/(m-2)) * (1/np.sqrt(T)) * np.sqrt((ma+mp)/(ma*mp)) * b0**2
    return(gamma)


def convert_hitran_error_to_uncertainty(gamma_err):
    gamma_err = gamma_err.replace(0, (.5))  # 0  ~~~  unreported or unavailable
    gamma_err = gamma_err.replace(1, (.5))  # 1  ~~~  Default or constant
    gamma_err = gamma_err.replace(2, (.5))  # 2  ~~~  Average or estimate
    gamma_err = gamma_err.replace(3, (.5))  # 3  ~~~  err >= 20 %              50
    gamma_err = gamma_err.replace(4, (.2))  # 4  ~~~  20 >= err >= 10 %        15
    gamma_err = gamma_err.replace(5, (.1))  # 5  ~~~  10 >= err >= 5 %         7.5
    gamma_err = gamma_err.replace(6, (.05))  # 6  ~~~  5 >= err >= 2 %          3.5
    gamma_err = gamma_err.replace(7, (.02))  # 7  ~~~  2 >= err >= 1 %          1.5
    gamma_err = gamma_err.replace(8, (.01))  # 8  ~~~  err <= 1 %               0.5

    return gamma_err


### Read in data

# import data

# absolute path to folder containing data
# rootdir_glob = '/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/hitran_data/**/*'
# be selective for data files
# file_list = [f for f in iglob(rootdir_glob, recursive=True) if os.path.isfile(f) if f[-10:] == "/1_iso.csv" if "readme" not in f]
# file_list =
# path = '/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/model_search/*'
# file_list = glob.glob(path + 'raw_data/*.csv')

path = '/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/Other_broadeners/Files_with_new_data/'
file_list = glob.glob(path + '*.csv')

# read data files, taking the filename from absolute path
db = {}
for f in file_list:
    i = f[101:104].strip('_')
    j = f[106:109].strip('_')
    if i == 'Bro' and j == 'eni':
        continue

    file = pd.read_csv(f, low_memory=False, dtype={'J': np.float64, 'Jpp': np.float64}, usecols=['J', 'Ka_aprox', 'Kc_aprox', 'Jpp', 'Kapp_aprox', 'Kcpp_aprox', 'T', 'profile', 'gamma', 'gamma_uncertainty',])
    file = file[file['profile'] == 'Voigt']
    if len(file) == 0:
        continue

    if i + j in db:
        db[i + '_' + j] = pd.concat([db[i + '_' + j], file]).reset_index(drop=True)
    else:
        db[i + '_' + j] = file.reset_index(drop=True)


print("HITRAN!!")

hit_file_list = glob.glob('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/model_search/raw_data/*.csv')
hit_db = {}
for f in hit_file_list:
    i = f[86:-4]
    print(i)
    if i != 'CH4':
        continue
    data_i = pd.read_csv(f, low_memory=False, dtype={'J': np.float64, 'Jpp': np.float64})
    gamma_cols = [col for col in data_i.columns if 'gamma' in col if 'err' not in col if 'ref' not in col]
    for broadener in gamma_cols:
        data_ij = data_i.copy()[['J', 'Ka_aprox', 'Kc_aprox', 'Jpp', 'Kapp_aprox', 'Kcpp_aprox', broadener, broadener+'-err']]
        j = broadener.replace('gamma_', '')
        good_data = data_ij[data_ij[broadener]!='#']
        print(j)
        print(len(good_data))
        if len(good_data) == 0:
            continue

        if j == 'self':
            j = i

        good_data['molecule'] = i
        good_data['broadener'] = j
        good_data['T'] = 296
        good_data['profile'] = 'Voigt'
        good_data[broadener+'-err'] = convert_hitran_error_to_uncertainty(good_data[broadener+'-err'])
        good_data = good_data.rename({broadener: 'gamma', broadener+'-err': 'gamma_uncertainty'}, axis=1)
        good_data['gamma'] = good_data['gamma'].astype(float)

        if i + j in db:
            db[i + '_' + j] = pd.concat([db[i + '_' + j], good_data]).reset_index(drop=True)
        else:
            db[i + '_' + j] = good_data.reset_index(drop=True)


file = pd.read_excel(path+'Broadening_data_from_smaller_sources.xlsx', dtype={'J':np.float64, 'Jpp':np.float64, 'gamma':np.float64, 'gamma_uncertainty':np.float64})
file = file[['molecule', 'broadener', 'J', 'Jpp', 'Ka_aprox', 'Kapp_aprox', 'Kc_aprox', 'Kcpp_aprox', 'gamma', 'gamma_uncertainty', 'n', 'T', 'profile', 'paper']].dropna()


file_mast = pd.read_excel('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/'+'Using LLMs to scrape data/Data_masters_student/data_Pengxia_21_Aug.xlsx', sheet_name=None)#, dtype={'J':np.float64, 'Jpp':np.float64, 'gamma':np.float64, 'gamma_uncertainty':np.float64}

count = 0
list_of_sheets = []
for sheet in file_mast.values():
    if set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]).issubset(set(sheet.columns)):
        print('One')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]]
        count+=1
    elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
        print('Two')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
        count+=1
    elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
        print('Three')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
        count+=1
    elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]).issubset(set(sheet.columns)):
        print('Four')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]]
        count+=1
    elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
        print('Five')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
        count+=1
    elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
        print('Six')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
        count+=1
    elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty (cm⁻¹ atm⁻¹)', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
        print('Seven')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty (cm⁻¹ atm⁻¹)', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
        count+=1
    elif count == 49:
        continue
    elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'.1"]).issubset(set(sheet.columns)):
        print('Eight')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'.1"]]
        count+=1
    elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature (K)', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]).issubset(set(sheet.columns)):
        print('Eight')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature (K)', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]]
        count+=1
    elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]).issubset(set(sheet.columns)):
        print('Eight')
        print(count)
        sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]]
        count+=1

    elif count == 157:
        continue
    elif count == 166:
        continue
    elif count == 168:
        continue
    elif count == 170:
        continue
    elif count == 183:
        continue
    elif count == 187:
        continue
    elif count == 193:
        continue
    elif count == 195:
        continue
    elif count == 204:
        continue
    elif count == 212:
        continue
    elif count == 200:
        continue
    elif count == 201:
        continue
    elif count == 209:
        continue

    print(count)
    print(sheet.columns)

    sheet = sheet.set_axis(['molecule', 'broadener', 'J', 'Jpp', 'gamma', 'gamma_uncertainty', 'gamma percent', 'n', 'T', 'profile', 'K', 'Ka_aprox', 'Kc_aprox', 'Kpp', 'Kapp_aprox', 'Kcpp_aprox'], axis=1)
    list_of_sheets.append(sheet)


masters_data = pd.concat(list_of_sheets)

masters_data['Ka_aprox'][masters_data['molecule'].str.len() == 2] = 0
masters_data['Kapp_aprox'][masters_data['molecule'].str.len() == 2] = 0
masters_data['Kc_aprox'][masters_data['molecule'].str.len() == 2] = masters_data['J']
masters_data['Kcpp_aprox'][masters_data['molecule'].str.len() == 2] = masters_data['Jpp']

masters_data = masters_data.drop(['K', 'Kpp', 'gamma percent', 'n'], axis=1)
masters_data['profile'] = 'Voigt'
masters_data = masters_data.dropna()
masters_data['paper'] = 9999

file = pd.concat([file, masters_data])

#create unique list of names
actives = file.molecule.unique()

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame() for elem in actives}

for key in DataFrameDict.keys():
    DataFrameDict[key] = file[:][file.molecule == key]

dfd3 = {}
for item, data in DataFrameDict.items():
    perts = data.broadener.unique()
    dfd2 = {elem : pd.DataFrame() for elem in perts}
    for key in dfd2.keys():
        dfd2[key] = data[:][data.broadener == key]
        dfd3[item+'_'+key] = dfd2[key]

for key, data in dfd3.items():
    data = data[data['profile'] == 'Voigt']
    if len(data) == 0:
        continue

    i, j = str.split(key, '_')
    print(i)
    print(j)
    print(data)
    if i+j in db:
        db[i+'_'+j] = pd.concat([db[i+'_'+j], data]).reset_index(drop=True)
    else:
        db[i+'_'+j] = data.reset_index(drop=True)



molecule_parameters = pd.read_csv('/Users/elizabeth/Desktop/line_broadening.nosync/line_broadening/molecule_parameters.csv', index_col=0)


HITRAN!!
CS2
HONO_prediction
CS
COF2
NO2
O3
HBr
O2
PH3
HNO3
N2
CF4
HO2
SO2
COCl2
SO3
HC3N
GeH4
CH3OH
OCS
HOBr
CH3I
H2O2
H2S
CH3Cl
HF
NH3
H2
CH3CN
SO
HI
ClO
CH3Br
C2N2
N2O
HOCl
S2
NF3
H2CO
SF6
CO2
H2O
CH3F
HCOOH
C2H2
CO
C2H4
OH
HCl
ClONO2
C4H2
HCN
C2H6
CH4
air
309863
self
309862


/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_26869/785149676.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_data['molecule'] = i
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_26869/785149676.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_data['broadener'] = j
/var/folders/rq/8prqh34x6vj4hqbm2_ssk_ch0000gn/T/ipykernel_26869/785149676.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

H2
0
He
0
CO2
0
H2O
309863
NO
One
0
1
Index(['active species', 'perturbing species', 'J'', 'J''',
       'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma',
       'percentage error in gamma', 'n (temperature exponent)', 'temperature',
       'line profile', 'K'', 'Ka'', 'Kc'', 'K''', 'Ka''', 'Kc''],
      dtype='object')
Two
1
2
Index(['active species', 'perturbing species', 'J'', 'J"',
       'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma',
       'percentage error in gamma', 'n (temperature exponent)', 'temperature',
       'line profile', 'K'', 'Ka'', 'Kc'', 'K"', 'Ka"', 'Kc"'],
      dtype='object')
Two
2
3
Index(['active species', 'perturbing species', 'J'', 'J"',
       'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma',
       'percentage error in gamma', 'n (temperature exponent)', 'temperature',
       'line profile', 'K'', 'Ka'', 'Kc'', 'K"', 'Ka"', 'Kc"'],
      dtype='object')
Two
3
4
Index(['active species', 'perturbing species', 'J'', 'J"',
       'gamma (cm⁻

In [111]:

masters_data['gamma_uncertainty'] = masters_data['gamma_uncertainty'].apply(pd.to_numeric, errors='coerce').astype(float).dropna()
masters_data['gamma'] = masters_data['gamma'].apply(pd.to_numeric, errors='coerce').astype(float).dropna()

ValueError: cannot reindex from a duplicate axis

In [112]:
masters_data

,molecule,broadener,J,Jpp,gamma,gamma_uncertainty,T,profile,Ka_aprox,Kc_aprox,Kapp_aprox,Kcpp_aprox,paper
0,CO2,H2O,4,5,0.08280,0.08280,296 K,Voigt,4,1,5,1,9999
1,CO2,H2O,4,5,0.09140,0.09140,296 K,Voigt,3,2,4,2,9999
2,CO2,H2O,3,4,0.09630,0.09630,296 K,Voigt,3,1,4,1,9999
3,CO2,H2O,3,4,0.09460,0.09460,296 K,Voigt,3,0,4,0,9999
4,CO2,H2O,3,4,0.10920,0.10920,296 K,Voigt,2,2,3,2,9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,O2,air,23,23,0.03781,0.00001,296,Voigt,0,23,0,23,9999
36,O2,air,25,25,0.03553,0.00008,296,Voigt,0,25,0,25,9999
37,O2,air,25,25,0.03563,0.00002,296,Voigt,0,25,0,25,9999
38,O2,air,27,27,0.03383,0.00005,296,Voigt,0,27,0,27,9999


In [110]:
masters_data['gamma_uncertainty'] / masters_data['gamma']

0     1.000000
1     1.000000
2     1.000000
3     1.000000
4     1.000000
        ...   
35    0.000264
36    0.002252
37    0.000561
38    0.001478
39    0.001267
Length: 3842, dtype: float64

In [ ]:
masters_data['profile'].unique()

In [ ]:
# count = 0
# list_of_sheets = []
# for sheet in file_mast.values():
#     if set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]).issubset(set(sheet.columns)):
#         print('One')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
#         print('Two')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
#         print('Three')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]).issubset(set(sheet.columns)):
#         print('Four')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
#         print('Five')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
#         print('Six')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty (cm⁻¹ atm⁻¹)', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']).issubset(set(sheet.columns)):
#         print('Seven')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", 'J"', 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty (cm⁻¹ atm⁻¹)', 'percentage error', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", 'K"', 'Ka"', 'Kc"']]
#         count+=1
#     elif count == 49:
#         continue
#     elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'.1"]).issubset(set(sheet.columns)):
#         print('Eight')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'.1"]]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature (K)', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]).issubset(set(sheet.columns)):
#         print('Eight')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'uncertainty in gamma (cm⁻¹ atm⁻¹)', 'percentage error in gamma', 'n (temperature exponent)', 'temperature (K)', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc''"]]
#         count+=1
#     elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]).issubset(set(sheet.columns)):
#         print('Eight')
#         print(count)
#         sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]]
#         count+=1
#     # elif set(['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma',
#     #    'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]).issubset(set(sheet.columns)):
#     #     print('Eight')
#     #     print(count)
#     #     sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm^-1 atm^-1)', 'absolute uncertainty in gamma',
#     #    'n (temperature exponent)', 'temperature', 'line profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]]
#     #     count+=1
#     elif count == 157:
#         continue
#     elif count == 166:
#         continue
#     elif count == 168:
#         continue
#     elif count == 170:
#         continue
#     elif count == 183:
#         continue
#     elif count == 187:
#         continue
#     elif count == 193:
#         continue
#     elif count == 195:
#         continue
#     elif count == 204:
#         continue
#     elif count == 212:
#         continue
#     elif count == 200:
#         continue
#     elif count == 201:
#         continue
#     elif count == 209:
#         continue
#
#     print(count)
#     print(sheet.columns)
#
#     sheet = sheet.set_axis(['molecule', 'broadener', 'J', 'Jpp', 'gamma', 'gamma_uncertainty', 'gamma percent', 'n', 'T', 'profile', 'K', 'Ka_aprox', 'Kc_aprox', 'Kpp', 'Kapp_aprox', 'Kcpp_aprox'], axis=1)
#     list_of_sheets.append(sheet)
#     #print(sheet)
#
#     #     sheet = sheet[['active species', 'perturbing species', "J'", "J''", 'gamma (cm⁻¹ atm⁻¹)', 'absolute uncertainty', 'percentage error in gamma', 'n (temperature exponent)', 'temperature', 'profile', "K'", "Ka'", "Kc'", "K''", "Ka''", "Kc'"]]
